# Calculate indicators

This notebook enables the user to use GDP, population, or other sets of indicators to calculate combined indicators such as per capita emissions or emissions / final energy use.

It should be used on data that is already pre-processed for this toolset to ensure efficiency and avoid errors.

One cell allows the user to adjust the units of the indicator. This should be done with care and the default option is not to do it! 

In [32]:
# import modules

# system
import sys, os, re
import time

# analytic
import pandas as pd
import numpy as np

# open cliamte data
import countrygroups

# plotting
import seaborn
import matplotlib.pyplot as plt

# global stocktake tools
import gst_tools.gst_utils as utils


In [25]:
# User options

data_set_1 = 'R-Andrew-2018_cement-CO2.csv'
data_set_2 = 'UN-2017-population.csv'

# Use this to generate the prefix of the output data file to include the source names of the original data. 
# The entities will automaticall be added when running the script
new_source_name = 'R-Andrew-2018-cement-CO2-UN-population'

# (don't need the file type ending!)



In [26]:
# get and clean data 

fname_in1 = os.path.join('proc-data', data_set_1)
fname_in2 = os.path.join('proc-data', data_set_2)

# read in the data
var1 = pd.read_csv(fname_in1)
var2 = pd.read_csv(fname_in2)

# make sure that the same countries and years are available
var1, var2 = utils.ensure_common_years(var1, var2)
var1, var2 = utils.ensure_common_countries(var1, var2)

# check the data format
check1 = utils.verify_data_format(var1)
check2 = utils.verify_data_format(var2)

if not check1 or not check2:
    print('One of the dataframes is not correct! Please check and try again!')
else:
    # get metadata for later use and checking
    var1_name = var1['variable'].unique()[0]
    var2_name = var2['variable'].unique()[0]

    var1_unit  = var1['unit'].unique()[0]
    var2_unit = var2['unit'].unique()[0]


Common countries are: 
['ESP', 'IRN', 'DZA', 'GAB', 'ZWE', 'PAN', 'NPL', 'HTI', 'JPN', 'UZB', 'IDN', 'LKA', 'SLV', 'FJI', 'MOZ', 'AUS', 'DNK', 'IND', 'ISL', 'AZE', 'AUT', 'BHS', 'ETH', 'HND', 'ZMB', 'PER', 'SEN', 'TZA', 'VEN', 'MDA', 'OMN', 'ISR', 'GEO', 'LBR', 'SAU', 'RUS', 'PHL', 'PRT', 'URY', 'MWI', 'KOR', 'MDG', 'DOM', 'CAN', 'FRA', 'ARE', 'COG', 'BRA', 'QAT', 'CRI', 'GTM', 'KHM', 'KWT', 'NIC', 'SDN', 'USA', 'KAZ', 'CPV', 'KEN', 'YEM', 'BWA', 'SRB', 'LUX', 'POL', 'FIN', 'MYS', 'NZL', 'UKR', 'ZAF', 'GRC', 'PRY', 'LAO', 'EGY', 'IRQ', 'PAK', 'TJK', 'CHL', 'DEU', 'EST', 'TTO', 'BFA', 'CHN', 'ERI', 'AGO', 'MNG', 'TGO', 'MAR', 'MEX', 'BIH', 'JAM', 'ECU', 'JOR', 'MLI', 'PNG', 'BGD', 'BGR', 'BRB', 'LBY', 'TUR', 'THA', 'NLD', 'NGA', 'AFG', 'COD', 'CMR', 'CYP', 'GHA', 'GIN', 'MMR', 'BHR', 'KGZ', 'SVN', 'UGA', 'SYR', 'SWE', 'ITA', 'TCD', 'SGP', 'ARG', 'ALB', 'VNM', 'TUN', 'CUB', 'RWA', 'CHE', 'COL', 'LVA', 'IRL', 'SUR', 'ROU', 'BEN', 'BOL', 'NER', 'BLR', 'SVK', 'NAM', 'PRK', 'LTU', 'HRV', 'BE

In [27]:
# combine data...

# for all of these, it's always var1 divided by var 2 and we want to ensure that this is done on countries. 
# Everything else should be consant across the table

def prep_df_for_division(df):
    
    df = df.set_index('country')
    
    year_cols = [y for y in df[df.columns] if (re.match(r"[0-9]{4,7}$", str(y)) is not None)]
    other_cols = list(set(df.columns) - set(year_cols))
    
    df = df.drop(other_cols, axis='columns')
    
    return df
    
# strip original metadata
var1 = prep_df_for_division(var1)
var2 = prep_df_for_division(var2)

# calculate new variables
new_df = var1 / var2

# generate new metadata
new_variable_name = var1_name + '-per-' + var2_name
new_df['variable'] = new_variable_name

# automatically generate the unit 
new_df['unit'] = var1_unit + ' / ' + var2_unit
    
new_df = new_df.reset_index()

# reorganise dataframe
new_df = utils.check_column_order(new_df)


In [28]:
# take a look at your new data frame

new_df

,country,unit,variable,1990,1991,1992,1993,1994,1995,1996,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,AFG,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.003669,0.003455,0.003421,0.003165,0.002951,0.002788,0.002671,...,0.000776,0.000425,0.000514,0.000401,0.000487,0.000468,0.000452,0.000620,0.001199,0.000937
1,AGO,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.008676,0.008404,0.009515,0.007625,0.007153,0.005668,0.007540,...,0.026640,0.025760,0.031453,0.030763,0.024751,0.023688,0.024422,0.029940,0.035984,0.035453
2,ALB,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.113882,0.091713,0.030528,0.031017,0.016334,0.031844,0.033180,...,0.085158,0.146598,0.153093,0.186759,0.220539,0.306732,0.375337,0.392946,0.375243,0.374912
3,ARE,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.730577,0.729940,0.749051,0.739330,0.869589,0.971488,0.930759,...,0.875233,0.958461,1.136305,0.891032,0.780226,0.734392,0.601542,0.607022,0.616038,0.610423
4,ARG,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.049893,0.046243,0.067776,0.074697,0.081920,0.070153,0.065049,...,0.095377,0.099699,0.097196,0.091374,0.099869,0.109202,0.099390,0.109474,0.104626,0.099867
5,ARM,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.103387,0.108723,0.027794,0.015872,0.009368,0.020564,0.023812,...,0.114315,0.138729,0.141736,0.083190,0.100232,0.091912,0.096429,0.094453,0.092422,0.091020
6,AUS,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.203211,0.184288,0.167163,0.169894,0.194825,0.185764,0.177542,...,0.188980,0.189624,0.181000,0.176135,0.160443,0.155517,0.154151,0.142285,0.133676,0.129246
7,AUT,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.263207,0.257954,0.268806,0.257473,0.264495,0.204127,0.203795,...,0.235855,0.256383,0.255803,0.214933,0.192867,0.196930,0.196418,0.193057,0.189848,0.195998
8,AZE,kt / ThousandPers,Cement-process-emissions-CO2-per-population,0.065997,0.063523,0.050911,0.039094,0.031131,0.013414,0.015225,...,0.090740,0.093589,0.087373,0.069584,0.068353,0.075232,0.102440,0.118164,0.145837,0.131531
9,BDI,kt / ThousandPers,Cement-process-emissions-CO2-per-population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.001555,0.003016,0.004174,0.002880,0.003990


In [29]:
# If desired, you can set the unit of the new data here. 

# ****** BE CAREFUL!!! *******
# This option is just to allow you to make the name nicer for plots and will not actually change the units. An 'org_unit' column will 
# be added to teh dataframe for safety. 
# Recommended option: set to False and keep automatic calculation of units.

convert_unit = True
desired_unit = 'ktCO2 / capita'
conversion_factor = 1000

# For reference: 
# * 1 Gg / Thousand Pers = 1 t / person

conv_df = new_df.copy()
org_unit = new_df['unit'].unique()

print('*******************')
print('Converting unit from "' + org_unit + '" to "' + desired_unit + 
      '" using a conversion factor of ' + str(conversion_factor))
print('*******************')

if convert_unit:
    conv_df['unit'] = desired_unit
    
    # convert the data
    conv_df, other_cols = utils.set_non_year_cols_as_index(conv_df)
    conv_df = conv_df * conversion_factor
    conv_df = conv_df.reset_index()

    final_df = conv_df
    
else: 
    
    final_df = new_df

*******************
['Converting unit from "kt / ThousandPers" to "ktCO2 / capita" using a conversion factor of 1000']
*******************


In [31]:
## write the data to file

"""
First ensure that years, unit, 'country', and variable are all in data. If they are
can proceed to print data
"""

if 'country' not in new_df.columns or 'unit' not in new_df.columns:
    
    print('Missing required information! Please check your input data and processing!')
    
else:
    
    # define filename as composite of variable and source name
    fname_out = new_source_name + '_' + new_variable_name + '.csv' 
    fullfname_out = os.path.join('proc-data', fname_out)

    # check folder exists
    if not os.path.exists('proc-data'):
        os.makedirs('proc-data')

    # check if file already exists
    files_present = glob.glob(fullname_out)

    # if no matching files, write to csv, if there are matching files, print statement
    if not files_present:
        # write to csv in proc data folder
        final_df.to_csv(fullfname_out, index=False)
    
    else:
        print 'WARNING: This file already exists! adding a date stamp to the file name.'
        fname_out = new_source_name + '_' + new_variable_name + time.strftime("%Y%m%d-%H") + '.csv' 
        final_df.to_csv(fullfname_out, index=False)

    # celebrate success 
    print('Processed data written to file!)
    print(fullfname_out)
    

Processed data written to file! - proc-data\R-Andrew-2018-cement-CO2-UN-population.csv_Cement-process-emissions-CO2-per-population.csv
